# 여러 datafield와 여러 instance를 각각 가장 최근에 업데이트된 1개 값만 남기기
NaN일 경우 계속 NaN으로 유지

In [1]:
import numpy as np
import pandas as pd

#### csv 읽는 경로 및 저장 경로만 수정 필요

In [2]:
# eid를 index_col로 지정 후, csv 읽기
df = pd.read_csv('ukbb/alcohol.csv', index_col='eid')
df.head()

,Unnamed: 0,1578-0.0,1578-1.0,1578-2.0,1578-3.0,1588-0.0,1588-1.0,1588-2.0,1588-3.0,4429-0.0,4429-1.0,4429-2.0,4429-3.0,4440-0.0,4440-1.0,4440-2.0,4440-3.0
eid,,,,,,,,,,,,,,,,,
1000015,0,2.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000027,1,-1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000039,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000040,3,3.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000053,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = df.drop(df.columns[0], axis=1)
df.head()

,1578-0.0,1578-1.0,1578-2.0,1578-3.0,1588-0.0,1588-1.0,1588-2.0,1588-3.0,4429-0.0,4429-1.0,4429-2.0,4429-3.0,4440-0.0,4440-1.0,4440-2.0,4440-3.0
eid,,,,,,,,,,,,,,,,
1000015,2.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000027,-1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000040,3.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# datafield만 생성
df_columns = list(df.columns)
data_field = list(set([i.split('-')[0] for i in df_columns]))
data_field

['1588', '4440', '4429', '1578']

In [5]:
# 길이가 datafield인 빈 list에 dataframe 추가하기
fields = [0] * len(data_field)
for i in range(len(data_field)):
    # 같은 datafield에 해당하는 column만 묶어서 list에 추가
    fields[i] = df.filter(like=data_field[i])

In [6]:
# 예시로 1588 datafield 출력
fields[0]

,1588-0.0,1588-1.0,1588-2.0,1588-3.0
eid,,,,
1000015,0.0,NaN,NaN,NaN
1000027,0.0,NaN,NaN,NaN
1000039,NaN,NaN,NaN,NaN
1000040,1.0,NaN,NaN,NaN
1000053,NaN,NaN,NaN,NaN
...,...,...,...,...
6023851,0.0,NaN,NaN,NaN
6023868,NaN,NaN,NaN,NaN
6023879,0.0,NaN,NaN,NaN


In [7]:
last_value_dfs = [0] * len(fields)
for i in range(len(fields)):
    # 각 datafield 별로 마지막으로 업데이트 된 값만 남기기 - 모두 다 Nan일 경우 계속 Nan으로 유지
    last_value_dfs[i] = fields[i].apply(lambda row: row[row.last_valid_index()] if not row.dropna().empty else None, axis=1)
    # series -> dataframe
    last_value_dfs[i] = last_value_dfs[i].to_frame()
    # column 명을 data field로 변경
    last_value_dfs[i].columns = [int(data_field[i])]

In [8]:
# 예시로 1588 datafield의 마지막 값 출력
last_value_dfs[0]

,1588
eid,
1000015,0.0
1000027,0.0
1000039,NaN
1000040,1.0
1000053,NaN
...,...
6023851,0.0
6023868,NaN
6023879,0.0


In [11]:
# concat 후 csv 저장
merged_df = pd.concat(last_value_dfs, axis = 1)
merged_df.to_csv('ukbb/alcohol_one_column.csv')
merged_df.head()

,1588,4440,4429,1578
eid,,,,
1000015,0.0,NaN,NaN,2.0
1000027,0.0,NaN,NaN,-1.0
1000039,NaN,NaN,NaN,NaN
1000040,1.0,NaN,NaN,3.0
1000053,NaN,NaN,NaN,NaN
